In [1]:
from training_pipe_GNN3_utils import TrainGNN3
from pathlib import Path
import argparse
import torch
import json
from visualize import plot_history_GNN3
import torch.multiprocessing as mp
import pyinstrument

In [13]:
import torch

# Votre tenseur à une dimension
tensor_1d = torch.tensor([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,0,1])

# Trouver l'indice du premier "1" après une séquence de "0" for all the tensor
split_index = (tensor_1d == 1).nonzero()

print(split_index)
# separate the tensor in as many part as we need
batch = torch.split(tensor_1d, split_index)

# Convertir la liste résultante en un tenseur de dimension 2

print(batch)

tensor([[ 3],
        [ 6],
        [ 7],
        [ 9],
        [12]])
5


TypeError: split_with_sizes(): argument 'split_sizes' (position 2) must be tuple of ints, not Tensor

In [51]:
# Votre tenseur à une dimension
tensor_1d = torch.tensor([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,0,0])

# Trouver les indices où le changement de 0 à 1 se produit
split_indices = (tensor_1d == 1).nonzero().flatten()


# Ajouter l'indice de fin du tenseur

# Calculer les longueurs des parties séparé
print(split_indices)
begin_indices_minus_1_flat = torch.cat([torch.tensor([-1]), split_indices[:-1]])
lengths = split_indices - begin_indices_minus_1_flat
print(split_indices - begin_indices_minus_1_flat)
print(lengths.squeeze(dim =1))
list_length = lengths.squeeze(dim =1).tolist()
# Répéter les indices pour obtenir les positions de début et de fin

# Utiliser les indices pour extraire les parties correspondantes du tenseur
batch = torch.split(tensor_1d, list_length)

print(batch)

tensor([ 3,  6,  7,  9, 12])
tensor([4, 3, 1, 2, 3])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [60]:
#problème plus compliqué, en fait il y a juste un 1 par batch qui nous interesse mais il n est pas forcément à la fin
# Votre tenseur à une dimension
tensor_1d = torch.tensor([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,0,0])

appartenance_tensor = torch.tensor([0,0,0,0,0,1,1,2,2,3,3,4,4,4,4])

# Trouver les indices où le changement de appartenance tensor prend + 1 se produit
begin_indices= torch.cat([torch.tensor([0]), (appartenance_tensor[1:] != appartenance_tensor[:-1]).nonzero().flatten() + 1])
print(begin_indices)


end_indices = torch.cat([begin_indices[1:],torch.tensor([len(appartenance_tensor)])])
print(end_indices)
lengths = end_indices - begin_indices
print(lengths)
list_length = lengths.tolist()
# Répéter les indices pour obtenir les positions de début et de fin

# Utiliser les indices pour extraire les parties correspondantes du tenseur
batch = torch.split(tensor_1d, list_length)

print(batch)

tensor([ 0,  5,  7,  9, 11])
tensor([ 5,  7,  9, 11, 15])
tensor([5, 2, 2, 2, 4])
(tensor([0, 0, 0, 1, 0]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([0, 1, 0, 0]))


In [2]:
CONFIG_PATH = Path('config_GNN3.json')

with open(CONFIG_PATH, 'r') as f:
        config = json.load(f)

In [3]:
# Call the train_GNN3 function with the provided arguments
mp.set_sharing_strategy('file_system') # Can cause memory leak

TrainingGNN3 = TrainGNN3(config)


Training on cuda
Loading data...
Dataset encoded with size 13
Dataset encoded with size 13
Data loaded
The 'GNN3_test_mem_leak' directory has been successfully created in the 'experiments' directory.


In [4]:
# Profile the training  

profiler = pyinstrument.Profiler()

profiler.start()
TrainingGNN3.train()
profiler.stop()

print(profiler.output_text(unicode=True, color=True))

Training: 100%|██████████| 1559/1559 [08:05<00:00,  3.21batch/s]

Evaluating: 100%|██████████| 195/195 [00:55<00:00,  3.52batch/s]

100%|██████████| 2/2 [15:38<00:00, 469.18s/it]



  _     ._   __/__   _ _  _  _ _/_   Recorded: 12:47:09  Samples:  785709
 /_//_/// /_\ / //_// / //_'/ //     Duration: 938.364   CPU time: 2147.641
/   _/                      v4.4.0

Program: c:\Users\goupi\.conda\envs\torch_geometric\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"84944315-617f-4a63-b1d9-f97d8ff7267a" --shell=9017 --transport="tcp" --iopub=9019 --f=c:\Users\goupi\AppData\Roaming\jupyter\runtime\kernel-v2-17988Q7cXCTYyzlDw.json

938.365 ZMQInteractiveShell.run_ast_nodes  IPython\core\interactiveshell.py:3301
└─ 938.359 <module>  ..\..\..\AppData\Local\Temp\ipykernel_23828\3376102744.py:1
   └─ 938.359 TrainGNN3.train  training_pipe_GNN3_utils.py:333
      ├─ 845.254 train_one_epoch  training_pipe_GNN3_utils.py:40
      │  ├─ 663.065 tqdm.__iter__  tqdm\std.py:1174
      │  │     [332 frames hidden]  tqdm, torch, <built-in>, ipykernel, t...
      │  │        649.170